In [71]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [72]:
df = pd.read_csv("Train.csv")
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [113]:
drop_cols = ['ID', 'Customer_care_calls', 'Cost_of_the_Product', 'Gender', 'Discount_offered', 'Weight_in_gms','Prior_purchases' ]  # ihtiyaca göre düzenle
df = df.drop(columns=[col for col in drop_cols if col in df.columns])
df

,Warehouse_block,Mode_of_Shipment,Customer_rating,Product_importance,Reached.on.Time_Y.N
0,D,Flight,2_musteri_puani_musteri_puani_musteri_puani,low_urun_onemi,1_zamaninda_ulasildi
1,F,Flight,5_musteri_puani_musteri_puani_musteri_puani,low_urun_onemi,1_zamaninda_ulasildi
2,A,Flight,2_musteri_puani_musteri_puani_musteri_puani,low_urun_onemi,1_zamaninda_ulasildi
3,B,Flight,3_musteri_puani_musteri_puani_musteri_puani,medium_urun_onemi,1_zamaninda_ulasildi
4,C,Flight,2_musteri_puani_musteri_puani_musteri_puani,medium_urun_onemi,1_zamaninda_ulasildi
...,...,...,...,...,...
10994,A,Ship,1_musteri_puani_musteri_puani_musteri_puani,medium_urun_onemi,1_zamaninda_ulasildi
10995,B,Ship,1_musteri_puani_musteri_puani_musteri_puani,medium_urun_onemi,0_gec_ulasildi
10996,C,Ship,4_musteri_puani_musteri_puani_musteri_puani,low_urun_onemi,0_gec_ulasildi
10997,F,Ship,2_musteri_puani_musteri_puani_musteri_puani,medium_urun_onemi,0_gec_ulasildi


In [103]:
df['Customer_rating'] = df['Customer_rating'].astype(str) + '_musteri_puani'

df['Product_importance'] = df['Product_importance'].replace({
    'low': 'low_urun_onemi',
    'medium': 'medium_urun_onemi',
    'high': 'high_urun_onemi'
})

df['Reached.on.Time_Y.N'] = df['Reached.on.Time_Y.N'].replace({
    1: '1_zamaninda_ulasildi',
    0: '0_gec_ulasildi'
})

In [114]:
df_encoded = pd.get_dummies(df)
df_encoded 

,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Customer_rating_1_musteri_puani_musteri_puani_musteri_puani,Customer_rating_2_musteri_puani_musteri_puani_musteri_puani,Customer_rating_3_musteri_puani_musteri_puani_musteri_puani,Customer_rating_4_musteri_puani_musteri_puani_musteri_puani,Customer_rating_5_musteri_puani_musteri_puani_musteri_puani,Product_importance_high_urun_onemi,Product_importance_low_urun_onemi,Product_importance_medium_urun_onemi,Reached.on.Time_Y.N_0_gec_ulasildi,Reached.on.Time_Y.N_1_zamaninda_ulasildi
0,False,False,False,True,False,True,False,False,False,True,False,False,False,False,True,False,False,True
1,False,False,False,False,True,True,False,False,False,False,False,False,True,False,True,False,False,True
2,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,True
3,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,True
4,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10994,True,False,False,False,False,False,False,True,True,False,False,False,False,False,False,True,False,True
10995,False,True,False,False,False,False,False,True,True,False,False,False,False,False,False,True,True,False
10996,False,False,True,False,False,False,False,True,False,False,False,True,False,False,True,False,True,False
10997,False,False,False,False,True,False,False,True,False,True,False,False,False,False,False,True,True,False


In [115]:
frequent_items = apriori(df_encoded, min_support=0.05, use_colnames=True)

In [116]:
rules = association_rules(frequent_items, metric="confidence", min_threshold=0.6)

In [117]:
def format_rules(rules_df):
    df_copy = rules_df.copy()
    df_copy['antecedents'] = df_copy['antecedents'].apply(lambda x: ', '.join(list(x)))
    df_copy['consequents'] = df_copy['consequents'].apply(lambda x: ', '.join(list(x)))
    return df_copy[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

In [118]:
#  SORU 1: Depo + Sevkiyat + Zamanında Ulaşma
# ------------------------------
rules_depo = rules[
    rules['antecedents'].apply(lambda x: 
        any('Warehouse_block_' in str(i) for i in x) and
        any('Mode_of_Shipment' in str(i) for i in x)
    )
    &
    rules['consequents'].apply(lambda x: any('1_zamaninda_ulasildi' in str(i) for i in x))
]
rules_depo_fmt = format_rules(rules_depo)
print("\n📦 Depo + Sevkiyat Kuralları:")
print(rules_depo_fmt.to_string(index=False))


📦 Depo + Sevkiyat Kuralları:
                                                                   antecedents                              consequents  support  confidence     lift
                                      Mode_of_Shipment_Ship, Warehouse_block_B Reached.on.Time_Y.N_1_zamaninda_ulasildi 0.068643    0.607401 1.017950
Mode_of_Shipment_Ship, Product_importance_medium_urun_onemi, Warehouse_block_F Reached.on.Time_Y.N_1_zamaninda_ulasildi 0.059278    0.602031 1.008951


In [110]:
#  SORU 2: Ürün Önemi + Sevkiyat + Zamanında Ulaşma
# ------------------------------
rules_urun = rules[
    rules['antecedents'].apply(lambda x: 
        any('urun_onemi' in str(i) for i in x) and
        any('Mode_of_Shipment_Ship' in str(i) for i in x)
    )
    &
    rules['consequents'].apply(lambda x: any('1_zamaninda_ulasildi' in str(i) for i in x))
]
rules_urun_fmt = format_rules(rules_urun)
print("\n📦 Ürün Önemi + Sevkiyat Kuralları:")
print(rules_urun_fmt.to_string(index=False))


📦 Ürün Önemi + Sevkiyat Kuralları:
                                                                   antecedents                              consequents  support  confidence     lift
Mode_of_Shipment_Ship, Product_importance_medium_urun_onemi, Warehouse_block_F Reached.on.Time_Y.N_1_zamaninda_ulasildi 0.059278    0.602031 1.008951


In [112]:
#  SORU 3: Müşteri Puanı + Zamanında Ulaşma
# ------------------------------
rules_musteri = rules[
    rules['antecedents'].apply(lambda x: 
        any('_musteri_puani' in str(i) for i in x)
    )
    &
    rules['consequents'].apply(lambda x: any('1_zamaninda_ulasildi' in str(i) for i in x))
]

rules_musteri_fmt = format_rules(rules_musteri)
print("\n📦 Müşteri Puanı Kuralları:")
print(rules_musteri_fmt.to_string(index=False))


📦 Müşteri Puanı Kuralları:
                                                                                      antecedents                              consequents  support  confidence     lift
                                      Customer_rating_3_musteri_puani_musteri_puani_musteri_puani Reached.on.Time_Y.N_1_zamaninda_ulasildi 0.123375    0.606074 1.015726
                                      Customer_rating_5_musteri_puani_musteri_puani_musteri_puani Reached.on.Time_Y.N_1_zamaninda_ulasildi 0.119738    0.606633 1.016662
               Mode_of_Shipment_Ship, Customer_rating_5_musteri_puani_musteri_puani_musteri_puani Reached.on.Time_Y.N_1_zamaninda_ulasildi 0.081735    0.613233 1.027724
Customer_rating_3_musteri_puani_musteri_puani_musteri_puani, Product_importance_medium_urun_onemi Reached.on.Time_Y.N_1_zamaninda_ulasildi 0.055732    0.612388 1.026307
   Product_importance_low_urun_onemi, Customer_rating_4_musteri_puani_musteri_puani_musteri_puani Reached.on.Time_Y.N_1_zamanin